In [1]:
# Loading all the libraries
import math as mt
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
import seaborn as sns
#Model Libraries
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

#Pre-Processing Libraries
#from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

#Evaluation Libraies
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.dummy import DummyClassifier
from sklearn.inspection import permutation_importance

#Warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Load the data files into different DataFrames
usage = pd.read_csv(r'‪C:\Users\HP\usage.csv')
#users = pd.read_csv(r'C:\Users\HP\megaline_users.csv')
#calls = pd.read_csv(r'C:\Users\HP\megaline_calls.csv')
#messages = pd.read_csv(r'C:\Users\HP\megaline_messages.csv')
#internet = pd.read_csv(r'C:\Users\HP\megaline_internet.csv')

In [3]:
# Print the general/summary information about the plans' DataFrame
usage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      int64  
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      int64  
 4   usd_per_gb             2 non-null      int64  
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
dtypes: float64(2), int64(5), object(1)
memory usage: 260.0+ bytes


In [4]:
# Rename existing column to new name keeping it in sync with data description
plans = plans.rename(columns={'usd_monthly_pay': 'usd_monthly_fee'})

In [5]:
# Get the list of columns names
plans.columns

Index(['messages_included', 'mb_per_month_included', 'minutes_included',
       'usd_monthly_fee', 'usd_per_gb', 'usd_per_message', 'usd_per_minute',
       'plan_name'],
      dtype='object')

In [6]:
# Print data for plans
plans

,messages_included,mb_per_month_included,minutes_included,usd_monthly_fee,usd_per_gb,usd_per_message,usd_per_minute,plan_name
0,50,15360,500,20,10,0.03,0.03,surf
1,1000,30720,3000,70,7,0.01,0.01,ultimate


In [7]:
# Calculate data volume allowance from mbs to gbs - 1 GB = 1024 megabytes
plans['gb_per_month_included'] = plans['mb_per_month_included'] / 1024

In [8]:
plans

,messages_included,mb_per_month_included,minutes_included,usd_monthly_fee,usd_per_gb,usd_per_message,usd_per_minute,plan_name,gb_per_month_included
0,50,15360,500,20,10,0.03,0.03,surf,15.0
1,1000,30720,3000,70,7,0.01,0.01,ultimate,30.0


In [9]:
# Print the general/summary information about the users' DataFrame
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    int64 
 1   first_name  500 non-null    object
 2   last_name   500 non-null    object
 3   age         500 non-null    int64 
 4   city        500 non-null    object
 5   reg_date    500 non-null    object
 6   plan        500 non-null    object
 7   churn_date  34 non-null     object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


In [10]:
plans

,messages_included,mb_per_month_included,minutes_included,usd_monthly_fee,usd_per_gb,usd_per_message,usd_per_minute,plan_name,gb_per_month_included
0,50,15360,500,20,10,0.03,0.03,surf,15.0
1,1000,30720,3000,70,7,0.01,0.01,ultimate,30.0


In [11]:
# Print a sample of data for users
users.sample(n=10, random_state=100)

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date
69,1069,Dino,Fry,31,"Houston-The Woodlands-Sugar Land, TX MSA",2018-09-17,ultimate,NaN
29,1029,Franklyn,Henson,59,"Tampa-St. Petersburg-Clearwater, FL MSA",2018-01-05,surf,NaN
471,1471,Britni,Mitchell,59,"Richmond, VA MSA",2018-10-13,surf,NaN
344,1344,Hugo,Daugherty,70,"New York-Newark-Jersey City, NY-NJ-PA MSA",2018-12-25,surf,NaN
54,1054,Denis,Frost,50,"Phoenix-Mesa-Chandler, AZ MSA",2018-04-21,surf,2018-12-31
189,1189,Terrence,Smith,75,"Columbus, OH MSA",2018-09-24,surf,NaN
463,1463,Dinorah,Simmons,30,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-11-27,ultimate,NaN
184,1184,Fredric,Middleton,39,"Sacramento–Roseville–Folsom, CA MSA",2018-03-07,ultimate,NaN
140,1140,Randolph,Graves,53,"New York-Newark-Jersey City, NY-NJ-PA MSA",2018-03-26,surf,NaN
125,1125,Assunta,Carver,39,"North Port-Sarasota-Bradenton, FL MSA",2018-08-16,surf,NaN


In [12]:
# Checking for duplicated user records
users.duplicated().sum()

0

In [13]:
# Checking for just duplicate user IDs
users['user_id'].duplicated().sum()

0

In [14]:
# Check plan column only contains the two specified plans
users['plan'].unique()

array(['ultimate', 'surf'], dtype=object)

In [15]:
# Check if city column has any duplicate values
sorted(users['city'].unique())

['Albany-Schenectady-Troy, NY MSA',
 'Albuquerque, NM MSA',
 'Atlanta-Sandy Springs-Roswell, GA MSA',
 'Austin-Round Rock-Georgetown, TX MSA',
 'Bakersfield, CA MSA',
 'Baltimore-Columbia-Towson, MD MSA',
 'Baton Rouge, LA MSA',
 'Birmingham-Hoover, AL MSA',
 'Boston-Cambridge-Newton, MA-NH MSA',
 'Bridgeport-Stamford-Norwalk, CT MSA',
 'Buffalo-Cheektowaga, NY MSA',
 'Cape Coral-Fort Myers, FL MSA',
 'Charleston-North Charleston, SC MSA',
 'Charlotte-Concord-Gastonia, NC-SC MSA',
 'Chicago-Naperville-Elgin, IL-IN-WI MSA',
 'Cincinnati, OH-KY-IN MSA',
 'Cleveland-Elyria, OH MSA',
 'Colorado Springs, CO MSA',
 'Columbia, SC MSA',
 'Columbus, OH MSA',
 'Dallas-Fort Worth-Arlington, TX MSA',
 'Dayton-Kettering, OH MSA',
 'Denver-Aurora-Lakewood, CO MSA',
 'Detroit-Warren-Dearborn, MI MSA',
 'El Paso, TX MSA',
 'Fresno, CA MSA',
 'Grand Rapids-Kentwood, MI MSA',
 'Greensboro-High Point, NC MSA',
 'Houston-The Woodlands-Sugar Land, TX MSA',
 'Indianapolis-Carmel-Anderson, IN MSA',
 'Jackson

In [16]:
# Convert reg_date to datetime format
users['reg_date'] = pd.to_datetime(users['reg_date'], format='%Y-%m-%d')

In [17]:
# Convert churn_date to datetime format
users['churn_date'] = pd.to_datetime(users['churn_date'], format='%Y-%m-%d')

In [18]:
users.dtypes

user_id                int64
first_name            object
last_name             object
age                    int64
city                  object
reg_date      datetime64[ns]
plan                  object
churn_date    datetime64[ns]
dtype: object

In [19]:
# Print a sample of data for users
users.sample(n=10, random_state=100)

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date
69,1069,Dino,Fry,31,"Houston-The Woodlands-Sugar Land, TX MSA",2018-09-17,ultimate,NaT
29,1029,Franklyn,Henson,59,"Tampa-St. Petersburg-Clearwater, FL MSA",2018-01-05,surf,NaT
471,1471,Britni,Mitchell,59,"Richmond, VA MSA",2018-10-13,surf,NaT
344,1344,Hugo,Daugherty,70,"New York-Newark-Jersey City, NY-NJ-PA MSA",2018-12-25,surf,NaT
54,1054,Denis,Frost,50,"Phoenix-Mesa-Chandler, AZ MSA",2018-04-21,surf,2018-12-31
189,1189,Terrence,Smith,75,"Columbus, OH MSA",2018-09-24,surf,NaT
463,1463,Dinorah,Simmons,30,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-11-27,ultimate,NaT
184,1184,Fredric,Middleton,39,"Sacramento–Roseville–Folsom, CA MSA",2018-03-07,ultimate,NaT
140,1140,Randolph,Graves,53,"New York-Newark-Jersey City, NY-NJ-PA MSA",2018-03-26,surf,NaT
125,1125,Assunta,Carver,39,"North Port-Sarasota-Bradenton, FL MSA",2018-08-16,surf,NaT


In [20]:
users['full_name'] = users['first_name'] + ' ' + users['last_name']

In [21]:
# Print a sample of data for users
users.sample(n=10, random_state=100)

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date,full_name
69,1069,Dino,Fry,31,"Houston-The Woodlands-Sugar Land, TX MSA",2018-09-17,ultimate,NaT,Dino Fry
29,1029,Franklyn,Henson,59,"Tampa-St. Petersburg-Clearwater, FL MSA",2018-01-05,surf,NaT,Franklyn Henson
471,1471,Britni,Mitchell,59,"Richmond, VA MSA",2018-10-13,surf,NaT,Britni Mitchell
344,1344,Hugo,Daugherty,70,"New York-Newark-Jersey City, NY-NJ-PA MSA",2018-12-25,surf,NaT,Hugo Daugherty
54,1054,Denis,Frost,50,"Phoenix-Mesa-Chandler, AZ MSA",2018-04-21,surf,2018-12-31,Denis Frost
189,1189,Terrence,Smith,75,"Columbus, OH MSA",2018-09-24,surf,NaT,Terrence Smith
463,1463,Dinorah,Simmons,30,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-11-27,ultimate,NaT,Dinorah Simmons
184,1184,Fredric,Middleton,39,"Sacramento–Roseville–Folsom, CA MSA",2018-03-07,ultimate,NaT,Fredric Middleton
140,1140,Randolph,Graves,53,"New York-Newark-Jersey City, NY-NJ-PA MSA",2018-03-26,surf,NaT,Randolph Graves
125,1125,Assunta,Carver,39,"North Port-Sarasota-Bradenton, FL MSA",2018-08-16,surf,NaT,Assunta Carver


In [22]:
# Print the general/summary information about the calls' DataFrame
calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         137735 non-null  object 
 1   user_id    137735 non-null  int64  
 2   call_date  137735 non-null  object 
 3   duration   137735 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.2+ MB


In [23]:
# Print a sample of data for calls
calls.sample(n=10, random_state=100)

,id,user_id,call_date,duration
72562,1257_302,1257,2018-11-15,20.74
88715,1324_773,1324,2018-06-23,7.91
44700,1161_115,1161,2018-12-09,5.79
81040,1291_640,1291,2018-04-29,14.45
9375,1042_305,1042,2018-05-05,12.28
107987,1382_984,1382,2018-10-19,12.47
112658,1397_265,1397,2018-10-19,13.18
117027,1408_676,1408,2018-04-19,12.23
13376,1057_255,1057,2018-10-25,0.00
2079,1009_501,1009,2018-11-13,0.00


In [24]:
# Checking for duplicated call records
calls.duplicated().sum()

0

In [25]:
# Checking for just duplicate call IDs
calls['id'].duplicated().sum()

0

In [26]:
# Convert call_date to datetime format
calls['call_date'] = pd.to_datetime(calls['call_date'], format='%Y-%m-%d')

In [27]:
calls.dtypes

id                   object
user_id               int64
call_date    datetime64[ns]
duration            float64
dtype: object

In [28]:
# Print a sample of data for calls
calls.sample(n=10, random_state=100)

,id,user_id,call_date,duration
72562,1257_302,1257,2018-11-15,20.74
88715,1324_773,1324,2018-06-23,7.91
44700,1161_115,1161,2018-12-09,5.79
81040,1291_640,1291,2018-04-29,14.45
9375,1042_305,1042,2018-05-05,12.28
107987,1382_984,1382,2018-10-19,12.47
112658,1397_265,1397,2018-10-19,13.18
117027,1408_676,1408,2018-04-19,12.23
13376,1057_255,1057,2018-10-25,0.00
2079,1009_501,1009,2018-11-13,0.00


In [29]:
# Check if all the records are from year 2018
calls['call_date'].dt.year.unique()

array([2018])

In [30]:
calls['rounded_up_duration'] = np.ceil(calls['duration'])

In [31]:
calls['call_month'] = calls['call_date'].dt.strftime('%b-%Y')

In [32]:
# Print a sample of data for calls
calls.sample(n=10, random_state=100)

,id,user_id,call_date,duration,rounded_up_duration,call_month
72562,1257_302,1257,2018-11-15,20.74,21.0,Nov-2018
88715,1324_773,1324,2018-06-23,7.91,8.0,Jun-2018
44700,1161_115,1161,2018-12-09,5.79,6.0,Dec-2018
81040,1291_640,1291,2018-04-29,14.45,15.0,Apr-2018
9375,1042_305,1042,2018-05-05,12.28,13.0,May-2018
107987,1382_984,1382,2018-10-19,12.47,13.0,Oct-2018
112658,1397_265,1397,2018-10-19,13.18,14.0,Oct-2018
117027,1408_676,1408,2018-04-19,12.23,13.0,Apr-2018
13376,1057_255,1057,2018-10-25,0.00,0.0,Oct-2018
2079,1009_501,1009,2018-11-13,0.00,0.0,Nov-2018


In [33]:
# Print a sample of data for calls
calls.sample(n=10, random_state=100)

,id,user_id,call_date,duration,rounded_up_duration,call_month
72562,1257_302,1257,2018-11-15,20.74,21.0,Nov-2018
88715,1324_773,1324,2018-06-23,7.91,8.0,Jun-2018
44700,1161_115,1161,2018-12-09,5.79,6.0,Dec-2018
81040,1291_640,1291,2018-04-29,14.45,15.0,Apr-2018
9375,1042_305,1042,2018-05-05,12.28,13.0,May-2018
107987,1382_984,1382,2018-10-19,12.47,13.0,Oct-2018
112658,1397_265,1397,2018-10-19,13.18,14.0,Oct-2018
117027,1408_676,1408,2018-04-19,12.23,13.0,Apr-2018
13376,1057_255,1057,2018-10-25,0.00,0.0,Oct-2018
2079,1009_501,1009,2018-11-13,0.00,0.0,Nov-2018


In [34]:
# Print a sample of data for calls
calls.sample(n=10, random_state=100)

,id,user_id,call_date,duration,rounded_up_duration,call_month
72562,1257_302,1257,2018-11-15,20.74,21.0,Nov-2018
88715,1324_773,1324,2018-06-23,7.91,8.0,Jun-2018
44700,1161_115,1161,2018-12-09,5.79,6.0,Dec-2018
81040,1291_640,1291,2018-04-29,14.45,15.0,Apr-2018
9375,1042_305,1042,2018-05-05,12.28,13.0,May-2018
107987,1382_984,1382,2018-10-19,12.47,13.0,Oct-2018
112658,1397_265,1397,2018-10-19,13.18,14.0,Oct-2018
117027,1408_676,1408,2018-04-19,12.23,13.0,Apr-2018
13376,1057_255,1057,2018-10-25,0.00,0.0,Oct-2018
2079,1009_501,1009,2018-11-13,0.00,0.0,Nov-2018


In [35]:
# Checking for duplicated call records
calls.duplicated().sum()

0

In [36]:
# Checking for just duplicate call IDs
calls['id'].duplicated().sum()

0

In [37]:
# Convert call_date to datetime format
calls['call_date'] = pd.to_datetime(calls['call_date'], format='%Y-%m-%d')

In [38]:
calls.dtypes

id                             object
user_id                         int64
call_date              datetime64[ns]
duration                      float64
rounded_up_duration           float64
call_month                     object
dtype: object

In [39]:
# Print a sample of data for calls
calls.sample(n=10, random_state=100)

,id,user_id,call_date,duration,rounded_up_duration,call_month
72562,1257_302,1257,2018-11-15,20.74,21.0,Nov-2018
88715,1324_773,1324,2018-06-23,7.91,8.0,Jun-2018
44700,1161_115,1161,2018-12-09,5.79,6.0,Dec-2018
81040,1291_640,1291,2018-04-29,14.45,15.0,Apr-2018
9375,1042_305,1042,2018-05-05,12.28,13.0,May-2018
107987,1382_984,1382,2018-10-19,12.47,13.0,Oct-2018
112658,1397_265,1397,2018-10-19,13.18,14.0,Oct-2018
117027,1408_676,1408,2018-04-19,12.23,13.0,Apr-2018
13376,1057_255,1057,2018-10-25,0.00,0.0,Oct-2018
2079,1009_501,1009,2018-11-13,0.00,0.0,Nov-2018


In [40]:
# Check if all the records are from year 2018
calls['call_date'].dt.year.unique()

array([2018])

In [41]:
calls['rounded_up_duration'] = np.ceil(calls['duration'])

In [42]:
calls['call_month'] = calls['call_date'].dt.strftime('%b-%Y')

In [43]:
calls['call_month'] = calls['call_date'].dt.strftime('%b-%Y')

In [44]:
# Print the general/summary information about the messages' DataFrame
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            76051 non-null  object
 1   user_id       76051 non-null  int64 
 2   message_date  76051 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [45]:
# Print a sample of data for messages
messages.sample(n=10, random_state=100)

,id,user_id,message_date
27652,1172_109,1172,2018-12-02
67387,1435_425,1435,2018-12-22
16140,1110_31,1110,2018-10-05
52094,1336_29,1336,2018-12-16
74767,1484_162,1484,2018-12-03
33895,1213_105,1213,2018-11-29
58754,1373_172,1373,2018-09-19
3321,1036_156,1036,2018-07-13
1653,1016_208,1016,2018-12-19
21376,1132_686,1132,2018-12-26


In [46]:
# Checking for duplicated message records
messages.duplicated().sum()

0

In [47]:
# Checking for just duplicate text message IDs
messages['id'].duplicated().sum()

0

In [48]:
# Convert message_date to datetime format
messages['message_date'] = pd.to_datetime(messages['message_date'], format='%Y-%m-%d')

In [49]:
messages.dtypes

id                      object
user_id                  int64
message_date    datetime64[ns]
dtype: object

In [50]:
# Print a sample of data for messages
messages.sample(n=10, random_state=100)

,id,user_id,message_date
27652,1172_109,1172,2018-12-02
67387,1435_425,1435,2018-12-22
16140,1110_31,1110,2018-10-05
52094,1336_29,1336,2018-12-16
74767,1484_162,1484,2018-12-03
33895,1213_105,1213,2018-11-29
58754,1373_172,1373,2018-09-19
3321,1036_156,1036,2018-07-13
1653,1016_208,1016,2018-12-19
21376,1132_686,1132,2018-12-26


In [51]:
# Check if all the records are from year 2018
messages['message_date'].dt.year.unique()

array([2018])

In [52]:
messages['message_month'] = messages['message_date'].dt.strftime('%b-%Y')

In [53]:
# Print a sample of data for messages
messages.sample(n=10, random_state=100)

,id,user_id,message_date,message_month
27652,1172_109,1172,2018-12-02,Dec-2018
67387,1435_425,1435,2018-12-22,Dec-2018
16140,1110_31,1110,2018-10-05,Oct-2018
52094,1336_29,1336,2018-12-16,Dec-2018
74767,1484_162,1484,2018-12-03,Dec-2018
33895,1213_105,1213,2018-11-29,Nov-2018
58754,1373_172,1373,2018-09-19,Sep-2018
3321,1036_156,1036,2018-07-13,Jul-2018
1653,1016_208,1016,2018-12-19,Dec-2018
21376,1132_686,1132,2018-12-26,Dec-2018


In [54]:
# Print the general/summary information about the internet DataFrame
internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104825 entries, 0 to 104824
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            104825 non-null  object 
 1   user_id       104825 non-null  int64  
 2   session_date  104825 non-null  object 
 3   mb_used       104825 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.2+ MB


In [55]:
# Print a sample of data for the internet traffic
internet.sample(n=10, random_state=100)

,id,user_id,session_date,mb_used
52071,1233_151,1233,2018-11-14,347.25
17527,1079_132,1079,2018-12-03,0.00
78045,1364_74,1364,2018-12-30,459.19
98810,1466_291,1466,2018-09-24,222.22
70473,1329_30,1329,2018-11-16,487.56
39942,1181_10,1181,2018-07-21,1145.38
103692,1496_0,1496,2018-10-19,348.64
97326,1458_405,1458,2018-10-22,0.00
18635,1083_87,1083,2018-11-24,328.03
98215,1465_28,1465,2018-11-20,625.68


In [56]:
# Checking for duplicated internet records
internet.duplicated().sum()

0

In [57]:
# Checking for just duplicate internet IDs
internet['id'].duplicated().sum()

0

In [58]:
# Convert session_date to datetime format
internet['session_date'] = pd.to_datetime(internet['session_date'], format='%Y-%m-%d')

In [59]:
internet.dtypes

id                      object
user_id                  int64
session_date    datetime64[ns]
mb_used                float64
dtype: object

In [60]:
# Print a sample of data for internet traffic
internet.sample(n=10, random_state=100)

,id,user_id,session_date,mb_used
52071,1233_151,1233,2018-11-14,347.25
17527,1079_132,1079,2018-12-03,0.00
78045,1364_74,1364,2018-12-30,459.19
98810,1466_291,1466,2018-09-24,222.22
70473,1329_30,1329,2018-11-16,487.56
39942,1181_10,1181,2018-07-21,1145.38
103692,1496_0,1496,2018-10-19,348.64
97326,1458_405,1458,2018-10-22,0.00
18635,1083_87,1083,2018-11-24,328.03
98215,1465_28,1465,2018-11-20,625.68


In [61]:
# Check if all the records are from year 2018
internet['session_date'].dt.year.unique()

array([2018])

In [62]:
# Calculate the volume of data spent during the session from mbs to gbs - 1 GB = 1024 megabytes
internet['gb_used'] = internet['mb_used'] / 1024

In [63]:
internet['session_month'] = internet['session_date'].dt.strftime('%b-%Y')

In [64]:
# Get list of colums
internet.columns

Index(['id', 'user_id', 'session_date', 'mb_used', 'gb_used', 'session_month'], dtype='object')

In [65]:
# Print a sample of data for internet traffic
internet.sample(n=10, random_state=100)

,id,user_id,session_date,mb_used,gb_used,session_month
52071,1233_151,1233,2018-11-14,347.25,0.339111,Nov-2018
17527,1079_132,1079,2018-12-03,0.00,0.000000,Dec-2018
78045,1364_74,1364,2018-12-30,459.19,0.448428,Dec-2018
98810,1466_291,1466,2018-09-24,222.22,0.217012,Sep-2018
70473,1329_30,1329,2018-11-16,487.56,0.476133,Nov-2018
39942,1181_10,1181,2018-07-21,1145.38,1.118535,Jul-2018
103692,1496_0,1496,2018-10-19,348.64,0.340469,Oct-2018
97326,1458_405,1458,2018-10-22,0.00,0.000000,Oct-2018
18635,1083_87,1083,2018-11-24,328.03,0.320342,Nov-2018
98215,1465_28,1465,2018-11-20,625.68,0.611016,Nov-2018


In [66]:
# Print out the plan conditions
plans

,messages_included,mb_per_month_included,minutes_included,usd_monthly_fee,usd_per_gb,usd_per_message,usd_per_minute,plan_name,gb_per_month_included
0,50,15360,500,20,10,0.03,0.03,surf,15.0
1,1000,30720,3000,70,7,0.01,0.01,ultimate,30.0


In [67]:
# Merge the users and calls Dataframes and print first 10 records
users_calls = users.merge(calls, on='user_id')
users_calls.head(10)

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date,full_name,id,call_date,duration,rounded_up_duration,call_month
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_93,2018-12-27,8.52,9.0,Dec-2018
1,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_145,2018-12-27,13.66,14.0,Dec-2018
2,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_247,2018-12-27,14.48,15.0,Dec-2018
3,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_309,2018-12-28,5.76,6.0,Dec-2018
4,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_380,2018-12-30,4.22,5.0,Dec-2018
5,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_388,2018-12-31,2.20,3.0,Dec-2018
6,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_510,2018-12-27,5.75,6.0,Dec-2018
7,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_521,2018-12-28,14.18,15.0,Dec-2018
8,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_530,2018-12-28,5.77,6.0,Dec-2018
9,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_544,2018-12-26,4.40,5.0,Dec-2018


In [68]:
# Calculate the number of calls made by each user per month. Save the result.

# Create a pivot table on user_id, full_name and call_month and count no. of unique call identifier - id
calls_per_user = users_calls.pivot_table(index=['user_id', 'full_name', 'call_month'], aggfunc={'id': 'count'})

# Give names to the columns of the the pivot table
calls_per_user.columns = ['number_of_calls']

In [69]:
# Print first 20 records
calls_per_user.head(20)

number_of_calls
user_id full_name        call_month                 
1000    Anamaria Bauer   Dec-2018                 16
1001    Mickey Wilkerson Aug-2018                 27
                         Dec-2018                 56
                         Nov-2018                 64
                         Oct-2018                 65
                         Sep-2018                 49
1002    Carlee Hoffman   Dec-2018                 47
                         Nov-2018                 55
                         Oct-2018                 11
1003    Reynaldo Jenkins Dec-2018                149
1004    Leonila Thompson Aug-2018                 49
                         Dec-2018                 50
                         Jul-2018                 49
                         Jun-2018                 44
                         May-2018                 21
                         Nov-2018                 54
                         Oct-2018                 61
                         Sep-2018                 42
1005    Livia Shields    Dec-2018                 59
1006    Jesusa Bradford  Dec-2018                  9

In [70]:
# Print last 20 records
calls_per_user.tail(20)

number_of_calls
user_id full_name       call_month                 
1496    Ariel Shepherd  Dec-2018                 40
                        Nov-2018                 39
                        Oct-2018                 46
                        Sep-2018                 52
1497    Donte Barrera   Dec-2018                 54
1498    Scot Williamson Apr-2018                 47
                        Aug-2018                 44
                        Dec-2018                 39
                        Feb-2018                 37
                        Jul-2018                 43
                        Jun-2018                 37
                        Mar-2018                 37
                        May-2018                 35
                        Nov-2018                 41
                        Oct-2018                 46
                        Sep-2018                 45
1499    Shena Dickson   Dec-2018                 65
                        Nov-2018                 45
                        Oct-2018                 53
                        Sep-2018                 41

In [71]:
users_calls.head(10)

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date,full_name,id,call_date,duration,rounded_up_duration,call_month
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_93,2018-12-27,8.52,9.0,Dec-2018
1,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_145,2018-12-27,13.66,14.0,Dec-2018
2,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_247,2018-12-27,14.48,15.0,Dec-2018
3,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_309,2018-12-28,5.76,6.0,Dec-2018
4,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_380,2018-12-30,4.22,5.0,Dec-2018
5,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_388,2018-12-31,2.20,3.0,Dec-2018
6,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_510,2018-12-27,5.75,6.0,Dec-2018
7,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_521,2018-12-28,14.18,15.0,Dec-2018
8,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_530,2018-12-28,5.77,6.0,Dec-2018
9,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_544,2018-12-26,4.40,5.0,Dec-2018


In [72]:
# Calculate the amount of minutes spent by each user per month. Save the result.

# Create a pivot table on user_id, full_name and call_month and sum duration and rounded_up_duration
minutes_per_user = users_calls.pivot_table(index=['user_id', 'full_name', 'call_month'], aggfunc={'duration': 'sum', 'rounded_up_duration': 'sum'})

# Give names to the columns of the the pivot table
minutes_per_user.columns = ['amount_of_actual_mins', 'amount_of_rounded_up_mins']

In [73]:
minutes_per_user.head(20)

amount_of_actual_mins  \
user_id full_name        call_month                          
1000    Anamaria Bauer   Dec-2018                   116.83   
1001    Mickey Wilkerson Aug-2018                   171.14   
                         Dec-2018                   392.93   
                         Nov-2018                   404.59   
                         Oct-2018                   374.11   
                         Sep-2018                   297.69   
1002    Carlee Hoffman   Dec-2018                   363.24   
                         Nov-2018                   359.76   
                         Oct-2018                    54.13   
1003    Reynaldo Jenkins Dec-2018                  1041.00   
1004    Leonila Thompson Aug-2018                   334.86   
                         Dec-2018                   403.53   
                         Jul-2018                   358.45   
                         Jun-2018                   261.32   
                         May-2018                   181.58   
                         Nov-2018                   452.98   
                         Oct-2018                   341.63   
                         Sep-2018                   284.60   
1005    Livia Shields    Dec-2018                   470.22   
1006    Jesusa Bradford  Dec-2018                    54.79   

                                     amount_of_rounded_up_mins  
user_id full_name        call_month                             
1000    Anamaria Bauer   Dec-2018                        124.0  
1001    Mickey Wilkerson Aug-2018                        182.0  
                         Dec-2018                        412.0  
                         Nov-2018                        426.0  
                         Oct-2018                        393.0  
                         Sep-2018                        315.0  
1002    Carlee Hoffman   Dec-2018                        384.0  
                         Nov-2018                        386.0  
                         Oct-2018                         59.0  
1003    Reynaldo Jenkins Dec-2018                       1104.0  
1004    Leonila Thompson Aug-2018                        354.0  
                         Dec-2018                        427.0  
                         Jul-2018                        381.0  
                         Jun-2018                        275.0  
                         May-2018                        193.0  
                         Nov-2018                        476.0  
                         Oct-2018                        365.0  
                         Sep-2018                        301.0  
1005    Livia Shields    Dec-2018                        496.0  
1006    Jesusa Bradford  Dec-2018                         59.0

In [74]:
minutes_per_user.tail(20)

amount_of_actual_mins  \
user_id full_name       call_month                          
1496    Ariel Shepherd  Dec-2018                   278.61   
                        Nov-2018                   291.88   
                        Oct-2018                   301.16   
                        Sep-2018                   389.94   
1497    Donte Barrera   Dec-2018                   276.53   
1498    Scot Williamson Apr-2018                   344.18   
                        Aug-2018                   244.57   
                        Dec-2018                   324.77   
                        Feb-2018                   231.87   
                        Jul-2018                   304.49   
                        Jun-2018                   225.57   
                        Mar-2018                   247.72   
                        May-2018                   275.13   
                        Nov-2018                   208.99   
                        Oct-2018                   278.06   
                        Sep-2018                   344.62   
1499    Shena Dickson   Dec-2018                   468.10   
                        Nov-2018                   288.56   
                        Oct-2018                   363.28   
                        Sep-2018                   330.37   

                                    amount_of_rounded_up_mins  
user_id full_name       call_month                             
1496    Ariel Shepherd  Dec-2018                        294.0  
                        Nov-2018                        308.0  
                        Oct-2018                        317.0  
                        Sep-2018                        414.0  
1497    Donte Barrera   Dec-2018                        300.0  
1498    Scot Williamson Apr-2018                        365.0  
                        Aug-2018                        266.0  
                        Dec-2018                        339.0  
                        Feb-2018                        244.0  
                        Jul-2018                        321.0  
                        Jun-2018                        238.0  
                        Mar-2018                        266.0  
                        May-2018                        288.0  
                        Nov-2018                        223.0  
                        Oct-2018                        298.0  
                        Sep-2018                        363.0  
1499    Shena Dickson   Dec-2018                        496.0  
                        Nov-2018                        308.0  
                        Oct-2018                        385.0  
                        Sep-2018                        346.0

In [75]:
minutes_per_user.tail(20)

amount_of_actual_mins  \
user_id full_name       call_month                          
1496    Ariel Shepherd  Dec-2018                   278.61   
                        Nov-2018                   291.88   
                        Oct-2018                   301.16   
                        Sep-2018                   389.94   
1497    Donte Barrera   Dec-2018                   276.53   
1498    Scot Williamson Apr-2018                   344.18   
                        Aug-2018                   244.57   
                        Dec-2018                   324.77   
                        Feb-2018                   231.87   
                        Jul-2018                   304.49   
                        Jun-2018                   225.57   
                        Mar-2018                   247.72   
                        May-2018                   275.13   
                        Nov-2018                   208.99   
                        Oct-2018                   278.06   
                        Sep-2018                   344.62   
1499    Shena Dickson   Dec-2018                   468.10   
                        Nov-2018                   288.56   
                        Oct-2018                   363.28   
                        Sep-2018                   330.37   

                                    amount_of_rounded_up_mins  
user_id full_name       call_month                             
1496    Ariel Shepherd  Dec-2018                        294.0  
                        Nov-2018                        308.0  
                        Oct-2018                        317.0  
                        Sep-2018                        414.0  
1497    Donte Barrera   Dec-2018                        300.0  
1498    Scot Williamson Apr-2018                        365.0  
                        Aug-2018                        266.0  
                        Dec-2018                        339.0  
                        Feb-2018                        244.0  
                        Jul-2018                        321.0  
                        Jun-2018                        238.0  
                        Mar-2018                        266.0  
                        May-2018                        288.0  
                        Nov-2018                        223.0  
                        Oct-2018                        298.0  
                        Sep-2018                        363.0  
1499    Shena Dickson   Dec-2018                        496.0  
                        Nov-2018                        308.0  
                        Oct-2018                        385.0  
                        Sep-2018                        346.0

In [76]:
# Merge the users and messages Dataframes and print first 10 records
users_messages = users.merge(messages, on='user_id')
users_messages.head(10)

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date,full_name,id,message_date,message_month
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_125,2018-12-27,Dec-2018
1,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_160,2018-12-31,Dec-2018
2,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_223,2018-12-31,Dec-2018
3,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_251,2018-12-27,Dec-2018
4,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_255,2018-12-26,Dec-2018
5,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_346,2018-12-29,Dec-2018
6,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_386,2018-12-30,Dec-2018
7,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_413,2018-12-31,Dec-2018
8,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_502,2018-12-27,Dec-2018
9,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_525,2018-12-28,Dec-2018


In [77]:
# Calculate the number of messages sent by each user per month. Save the result.

# Create a pivot table on user_id, full_name and message_month and count no. of unique text message identifier - id
messages_per_user = users_messages.pivot_table(index=['user_id', 'full_name', 'message_month'], aggfunc={'id': 'count'})

# Give names to the columns of the the pivot table
messages_per_user.columns = ['number_of_messages']

In [78]:
messages_per_user.head(20)

number_of_messages
user_id full_name        message_month                    
1000    Anamaria Bauer   Dec-2018                       11
1001    Mickey Wilkerson Aug-2018                       30
                         Dec-2018                       44
                         Nov-2018                       36
                         Oct-2018                       53
                         Sep-2018                       44
1002    Carlee Hoffman   Dec-2018                       41
                         Nov-2018                       32
                         Oct-2018                       15
1003    Reynaldo Jenkins Dec-2018                       50
1004    Leonila Thompson Aug-2018                       25
                         Dec-2018                       31
                         Jul-2018                       26
                         Jun-2018                       18
                         May-2018                        7
                         Nov-2018                       25
                         Oct-2018                       24
                         Sep-2018                       21
1005    Livia Shields    Dec-2018                       11
1006    Jesusa Bradford  Dec-2018                      139

In [79]:
messages_per_user.tail(20)

number_of_messages
user_id full_name      message_month                    
1491    Terina Jensen  May-2018                       45
                       Nov-2018                       46
                       Oct-2018                       51
                       Sep-2018                       50
1492    Elvie Jones    Dec-2018                       37
                       Nov-2018                       31
                       Oct-2018                       29
                       Sep-2018                       11
1494    Preston Downs  Aug-2018                       27
                       Dec-2018                       33
                       Jul-2018                       20
                       Nov-2018                       35
                       Oct-2018                       38
                       Sep-2018                       21
1496    Ariel Shepherd Aug-2018                        2
                       Dec-2018                       11
                       Nov-2018                       13
                       Oct-2018                       18
                       Sep-2018                       21
1497    Donte Barrera  Dec-2018                       50

In [80]:
# Merge the users and internet Dataframes and print first 10 records
users_internet = users.merge(internet, on='user_id')
users_internet.head(10)

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date,full_name,id,session_date,mb_used,gb_used,session_month
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_13,2018-12-29,89.86,0.087754,Dec-2018
1,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_204,2018-12-31,0.00,0.000000,Dec-2018
2,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_379,2018-12-28,660.40,0.644922,Dec-2018
3,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_413,2018-12-26,270.99,0.264639,Dec-2018
4,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaT,Anamaria Bauer,1000_442,2018-12-27,880.22,0.859590,Dec-2018
5,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaT,Mickey Wilkerson,1001_0,2018-08-24,284.68,0.278008,Aug-2018
6,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaT,Mickey Wilkerson,1001_3,2018-12-09,656.04,0.640664,Dec-2018
7,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaT,Mickey Wilkerson,1001_4,2018-11-04,16.97,0.016572,Nov-2018
8,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaT,Mickey Wilkerson,1001_10,2018-11-27,135.18,0.132012,Nov-2018
9,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaT,Mickey Wilkerson,1001_15,2018-12-13,761.92,0.744062,Dec-2018


In [81]:
# Calculate the volume of internet traffic used by each user per month. Save the result.

# Create a pivot table on user_id, full_name and session_month and apply sum on gb_used
internet_traffic_per_user = users_internet.pivot_table(index=['user_id', 'full_name', 'session_month'], aggfunc={'gb_used': 'sum'})

# Give names to the columns of the the pivot table
internet_traffic_per_user.columns = ['actual_gb_used']

In [82]:
internet_traffic_per_user.head(10)

actual_gb_used
user_id full_name        session_month                
1000    Anamaria Bauer   Dec-2018             1.856904
1001    Mickey Wilkerson Aug-2018             6.756982
                         Dec-2018            18.915215
                         Nov-2018            18.070605
                         Oct-2018            21.807119
                         Sep-2018            13.002754
1002    Carlee Hoffman   Dec-2018            14.058828
                         Nov-2018            18.891680
                         Oct-2018             6.398447
1003    Reynaldo Jenkins Dec-2018            26.410293

In [83]:
internet_traffic_per_user.tail(20)

actual_gb_used
user_id full_name       session_month                
1496    Ariel Shepherd  Dec-2018            16.053701
                        Nov-2018             8.347031
                        Oct-2018            13.952500
                        Sep-2018            16.005146
1497    Donte Barrera   Dec-2018            10.846240
1498    Scot Williamson Apr-2018            19.412949
                        Aug-2018            19.787002
                        Dec-2018            22.595400
                        Feb-2018            19.357461
                        Jul-2018            24.252314
                        Jun-2018            14.460137
                        Mar-2018            19.281582
                        May-2018            21.935713
                        Nov-2018            18.719287
                        Oct-2018            20.098398
                        Sep-2018            22.292266
1499    Shena Dickson   Dec-2018            21.542197
                        Nov-2018            16.419756
                        Oct-2018            19.035576
                        Sep-2018            12.680430

In [84]:
internet_traffic_per_user['rounded_up_gb_used'] = np.ceil(internet_traffic_per_user['actual_gb_used'])
internet_traffic_per_user.head(20)

actual_gb_used  rounded_up_gb_used
user_id full_name        session_month                                    
1000    Anamaria Bauer   Dec-2018             1.856904                 2.0
1001    Mickey Wilkerson Aug-2018             6.756982                 7.0
                         Dec-2018            18.915215                19.0
                         Nov-2018            18.070605                19.0
                         Oct-2018            21.807119                22.0
                         Sep-2018            13.002754                14.0
1002    Carlee Hoffman   Dec-2018            14.058828                15.0
                         Nov-2018            18.891680                19.0
                         Oct-2018             6.398447                 7.0
1003    Reynaldo Jenkins Dec-2018            26.410293                27.0
1004    Leonila Thompson Aug-2018            27.325918                28.0
                         Dec-2018            20.887979                21.0
                         Jul-2018            23.942012                24.0
                         Jun-2018            20.188301                21.0
                         May-2018             6.393760                 7.0
                         Nov-2018            21.338652                22.0
                         Oct-2018            14.200811                15.0
                         Sep-2018            18.410859                19.0
1005    Livia Shields    Dec-2018            16.738447                17.0
1006    Jesusa Bradford  Dec-2018            31.366035                32.0

In [85]:
# Flatten calls_per_user Dataframe
calls_per_user = calls_per_user.reset_index()
calls_per_user.head(10)

,user_id,full_name,call_month,number_of_calls
0,1000,Anamaria Bauer,Dec-2018,16
1,1001,Mickey Wilkerson,Aug-2018,27
2,1001,Mickey Wilkerson,Dec-2018,56
3,1001,Mickey Wilkerson,Nov-2018,64
4,1001,Mickey Wilkerson,Oct-2018,65
5,1001,Mickey Wilkerson,Sep-2018,49
6,1002,Carlee Hoffman,Dec-2018,47
7,1002,Carlee Hoffman,Nov-2018,55
8,1002,Carlee Hoffman,Oct-2018,11
9,1003,Reynaldo Jenkins,Dec-2018,149


In [86]:
# Flatten minutes_per_user Dataframe
minutes_per_user = minutes_per_user.reset_index()
minutes_per_user.head(10)

,user_id,full_name,call_month,amount_of_actual_mins,amount_of_rounded_up_mins
0,1000,Anamaria Bauer,Dec-2018,116.83,124.0
1,1001,Mickey Wilkerson,Aug-2018,171.14,182.0
2,1001,Mickey Wilkerson,Dec-2018,392.93,412.0
3,1001,Mickey Wilkerson,Nov-2018,404.59,426.0
4,1001,Mickey Wilkerson,Oct-2018,374.11,393.0
5,1001,Mickey Wilkerson,Sep-2018,297.69,315.0
6,1002,Carlee Hoffman,Dec-2018,363.24,384.0
7,1002,Carlee Hoffman,Nov-2018,359.76,386.0
8,1002,Carlee Hoffman,Oct-2018,54.13,59.0
9,1003,Reynaldo Jenkins,Dec-2018,1041.00,1104.0


In [87]:
# Merge the data for calls and minutes based on user_id and month and save it in - user_consumption_per_month
user_consumption_per_month = calls_per_user.merge(minutes_per_user, how='outer', on=['user_id', 'call_month'])

In [88]:
# Create a column - user_name and save value from full_name_x. If that is null, then get value from full_name_y
user_consumption_per_month['user_name'] = user_consumption_per_month['full_name_x'].fillna(user_consumption_per_month['full_name_y'])

# Get only the necessary columns from user_consumption_per_month
user_consumption_per_month = user_consumption_per_month[['user_id', 'user_name', 'call_month', 'number_of_calls', 'amount_of_actual_mins', 'amount_of_rounded_up_mins']]
user_consumption_per_month

# Rename columns more meaningfully in user_consumption_per_month
user_consumption_per_month = user_consumption_per_month.rename(columns={'call_month': 'month', 'amount_of_actual_mins': 'call_mins', 'amount_of_rounded_up_mins': 'rounded_up_call_mins' })

In [89]:
# Get a sample of 20 records
user_consumption_per_month.sample(n=20, random_state=100)

,user_id,user_name,month,number_of_calls,call_mins,rounded_up_call_mins
670,1145,Venice Brooks,Jul-2018,54,323.43,350.0
1235,1264,Jessie Hill,Nov-2018,62,398.01,422.0
589,1127,Marchelle Brock,Jul-2018,35,231.33,241.0
977,1211,Vito Cameron,Aug-2018,42,352.36,371.0
63,1017,Boris Gates,Dec-2018,70,450.86,476.0
1059,1231,Gaylord Witt,Jul-2018,77,496.50,526.0
1387,1309,Winfred Zimmerman,Nov-2018,3,21.71,23.0
231,1054,Denis Frost,Sep-2018,41,305.32,326.0
1461,1325,Cleora Lyons,Dec-2018,67,401.76,429.0
1664,1365,Milford Rush,Dec-2018,180,1283.20,1357.0


In [90]:
# Flatten messages_per_user Dataframe
messages_per_user = messages_per_user.reset_index()
messages_per_user.head(20)

,user_id,full_name,message_month,number_of_messages
0,1000,Anamaria Bauer,Dec-2018,11
1,1001,Mickey Wilkerson,Aug-2018,30
2,1001,Mickey Wilkerson,Dec-2018,44
3,1001,Mickey Wilkerson,Nov-2018,36
4,1001,Mickey Wilkerson,Oct-2018,53
5,1001,Mickey Wilkerson,Sep-2018,44
6,1002,Carlee Hoffman,Dec-2018,41
7,1002,Carlee Hoffman,Nov-2018,32
8,1002,Carlee Hoffman,Oct-2018,15
9,1003,Reynaldo Jenkins,Dec-2018,50


In [91]:
# Merge the data for messages and user_consumption_per_month based on user_id and month and save it in - user_consumption_per_month
user_consumption_per_month = user_consumption_per_month.merge(messages_per_user, how='outer', left_on=['user_id', 'month'], right_on=['user_id', 'message_month'])

In [92]:
# Save value from user_name. If that is null, then get value from full_name
user_consumption_per_month['user_name'] = user_consumption_per_month['user_name'].fillna(user_consumption_per_month['full_name'])

# Save value from month. If that is null, then get value from message_month
user_consumption_per_month['month'] = user_consumption_per_month['month'].fillna(user_consumption_per_month['message_month'])

# Get only the necessary columns from user_consumption_per_month
user_consumption_per_month = user_consumption_per_month[['user_id', 'user_name', 'month', 'number_of_calls', 'call_mins', 'rounded_up_call_mins', 'number_of_messages']]

In [93]:
# Get a sample of 20 records
user_consumption_per_month.sample(n=20, random_state=800)

,user_id,user_name,month,number_of_calls,call_mins,rounded_up_call_mins,number_of_messages
2269,1131,Shane Morrison,Jul-2018,NaN,NaN,NaN,28.0
295,1065,Sherlyn Noel,Nov-2018,80.0,596.19,626.0,29.0
1122,1246,Stevie Moran,Apr-2018,104.0,660.03,699.0,19.0
1615,1358,Vaughn Maldonado,Aug-2018,68.0,498.60,528.0,4.0
1340,1294,Demetrius Russo,Sep-2018,17.0,136.91,144.0,14.0
955,1203,Manie Grimes,Nov-2018,64.0,366.48,394.0,47.0
860,1184,Fredric Middleton,Dec-2018,90.0,630.58,666.0,37.0
290,1064,Gerardo Ashley,Jun-2018,24.0,142.45,153.0,3.0
21,1007,Eusebio Welch,Aug-2018,70.0,427.95,456.0,51.0
296,1065,Sherlyn Noel,Oct-2018,87.0,687.97,723.0,29.0


In [94]:
# Flatten internet_traffic_per_user Dataframe
internet_traffic_per_user = internet_traffic_per_user.reset_index()
internet_traffic_per_user.head(20)

,user_id,full_name,session_month,actual_gb_used,rounded_up_gb_used
0,1000,Anamaria Bauer,Dec-2018,1.856904,2.0
1,1001,Mickey Wilkerson,Aug-2018,6.756982,7.0
2,1001,Mickey Wilkerson,Dec-2018,18.915215,19.0
3,1001,Mickey Wilkerson,Nov-2018,18.070605,19.0
4,1001,Mickey Wilkerson,Oct-2018,21.807119,22.0
5,1001,Mickey Wilkerson,Sep-2018,13.002754,14.0
6,1002,Carlee Hoffman,Dec-2018,14.058828,15.0
7,1002,Carlee Hoffman,Nov-2018,18.891680,19.0
8,1002,Carlee Hoffman,Oct-2018,6.398447,7.0
9,1003,Reynaldo Jenkins,Dec-2018,26.410293,27.0


In [95]:
# Merge the data for internet and user_consumption_per_month based on user_id and month and save it in - user_consumption_per_month
user_consumption_per_month = user_consumption_per_month.merge(internet_traffic_per_user, how='outer', left_on=['user_id', 'month'], right_on=['user_id', 'session_month'])

In [96]:
# Save value from user_name. If that is null, then get value from full_name
user_consumption_per_month['user_name'] = user_consumption_per_month['user_name'].fillna(user_consumption_per_month['full_name'])

# Save value from month. If that is null, then get value from session_month
user_consumption_per_month['month'] = user_consumption_per_month['month'].fillna(user_consumption_per_month['session_month'])

# Get only the necessary columns from user_consumption_per_month
user_consumption_per_month = user_consumption_per_month[['user_id', 'user_name', 'month', 'number_of_calls', 'call_mins', 'rounded_up_call_mins', 'number_of_messages', 'actual_gb_used', 'rounded_up_gb_used']]

In [97]:
# Get a sample of 20 records
user_consumption_per_month.sample(n=20, random_state=989)

,user_id,user_name,month,number_of_calls,call_mins,rounded_up_call_mins,number_of_messages,actual_gb_used,rounded_up_gb_used
1705,1374,Ching Watts,Dec-2018,51.0,252.79,272.0,83.0,29.071426,30.0
2235,1495,Fidel Sharpe,Oct-2018,75.0,578.03,610.0,NaN,26.268955,27.0
357,1076,Ronny Browning,Nov-2018,57.0,362.29,383.0,139.0,20.818984,21.0
1114,1244,Demarcus Ellis,Dec-2018,19.0,144.49,151.0,41.0,19.073398,20.0
1529,1337,Lucius Arnold,Nov-2018,55.0,344.94,370.0,46.0,12.989189,13.0
378,1079,Brian Mccall,Oct-2018,108.0,693.16,732.0,71.0,16.968652,17.0
528,1113,Spencer Powers,Nov-2018,71.0,525.60,553.0,30.0,12.479219,13.0
825,1176,Terrance Shaffer,Dec-2018,68.0,404.47,432.0,76.0,25.203359,26.0
2252,1498,Scot Williamson,Oct-2018,46.0,278.06,298.0,NaN,20.098398,21.0
1118,1245,Quinn David,Jul-2018,10.0,58.71,64.0,4.0,2.471523,3.0


In [98]:
# Merge users and plans dataframes on plan or plan_name
users_plans = users[['user_id', 'plan']].merge(plans, left_on='plan', right_on='plan_name' )

# Take out the redundant column for plan
users_plans = users_plans.loc[:, users_plans.columns != 'plan_name']

In [99]:
# Get random 20 records from users_plans
users_plans.sample(n=20, random_state=200)

,user_id,plan,messages_included,mb_per_month_included,minutes_included,usd_monthly_fee,usd_per_gb,usd_per_message,usd_per_minute,gb_per_month_included
157,1487,ultimate,1000,30720,3000,70,7,0.01,0.01,30.0
88,1279,ultimate,1000,30720,3000,70,7,0.01,0.01,30.0
383,1321,surf,50,15360,500,20,10,0.03,0.03,15.0
125,1409,ultimate,1000,30720,3000,70,7,0.01,0.01,30.0
445,1410,surf,50,15360,500,20,10,0.03,0.03,15.0
13,1037,ultimate,1000,30720,3000,70,7,0.01,0.01,30.0
438,1398,surf,50,15360,500,20,10,0.03,0.03,15.0
374,1309,surf,50,15360,500,20,10,0.03,0.03,15.0
235,1111,surf,50,15360,500,20,10,0.03,0.03,15.0
256,1140,surf,50,15360,500,20,10,0.03,0.03,15.0


In [100]:
# Add the plan information
user_consumption_per_month = user_consumption_per_month.merge(users_plans, on='user_id')

In [101]:
# Get random 20 records from user_consumption_per_month
user_consumption_per_month.sample(n=20, random_state=200)

,user_id,user_name,month,number_of_calls,call_mins,rounded_up_call_mins,number_of_messages,actual_gb_used,rounded_up_gb_used,plan,messages_included,mb_per_month_included,minutes_included,usd_monthly_fee,usd_per_gb,usd_per_message,usd_per_minute,gb_per_month_included
143,1039,Flo Hicks,Oct-2018,59.0,431.84,457.0,23.0,13.194590,14.0,ultimate,1000,30720,3000,70,7,0.01,0.01,30.0
1104,1240,Drema Lopez,Oct-2018,162.0,1129.04,1194.0,9.0,41.141445,42.0,surf,50,15360,500,20,10,0.03,0.03,15.0
670,1145,Venice Brooks,Dec-2018,55.0,340.36,364.0,61.0,18.362188,19.0,surf,50,15360,500,20,10,0.03,0.03,15.0
1965,1427,Zofia Brock,Oct-2018,38.0,214.77,229.0,NaN,12.340439,13.0,ultimate,1000,30720,3000,70,7,0.01,0.01,30.0
2106,1465,Arianna Morrison,Jun-2018,30.0,177.89,189.0,16.0,1.865693,2.0,ultimate,1000,30720,3000,70,7,0.01,0.01,30.0
1323,1291,Angeles Mejia,Jul-2018,79.0,611.52,642.0,10.0,20.993428,21.0,surf,50,15360,500,20,10,0.03,0.03,15.0
494,1106,Eddy Garrison,Jun-2018,77.0,473.02,506.0,NaN,18.180664,19.0,surf,50,15360,500,20,10,0.03,0.03,15.0
1226,1263,Lucio Duffy,Jun-2018,81.0,695.60,733.0,6.0,15.320820,16.0,surf,50,15360,500,20,10,0.03,0.03,15.0
1081,1236,Odell Juarez,Apr-2018,49.0,251.98,273.0,28.0,15.238115,16.0,ultimate,1000,30720,3000,70,7,0.01,0.01,30.0
1368,1301,Tomas Horton,Nov-2018,23.0,147.15,155.0,33.0,8.739062,9.0,ultimate,1000,30720,3000,70,7,0.01,0.01,30.0
